In [74]:
import os
import psutil
import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom
from pandas import read_csv
import statsmodels.api as sm
import time
import pm4py
from datetime import datetime
from datetime import date
from datetime import datetime
from datetime import timedelta

from numba import jit

from sklearn.linear_model import LinearRegression  # for linear regression
from sklearn import linear_model
from sklearn.cluster import KMeans  # for clustering
from sklearn.tree import DecisionTreeClassifier  # for decision tree mining
from sklearn.metrics import mean_absolute_error, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf 
import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot
from matplotlib import pyplot
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

from tensorflow import keras
from sklearn.linear_model import HuberRegressor

In [78]:
file_export = 'export2018.csv'
data = pd.read_csv(file_export)

# Pre-processing

Pre-processing
* Visualization
1. Unix time
2. Encoding of categorical features
3. Temporal ordering
4. Aditional features:
- Previous event
- Next event
- Day of the week
- Time of day
- Event duration
5. Separate 80-20 
- Visualization
6. Get rid of overlap

In [79]:
data = data.sort_values(by=['case','startTime'])

In [80]:
#Duration
@jit(parallel = True)
def calculator_nb(case, startTime):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break

        if (case[idx + 1] == case[idx]):
            res[idx] = startTime[idx + 1]
        else:
            res[idx] = startTime[idx]

        idx+=1
    return res

data['completeTime'] = calculator_nb(data['case'].values, data['startTime'].values)
data.at[317373, 'completeTime'] = data.at[317373, 'startTime']

data['startTime'] =  pd.to_datetime(data['startTime'])
data['completeTime'] =  pd.to_datetime(data['completeTime'])
data['duration'] = data['completeTime'] - data['startTime']
#to turn duration into seconds:
duration = data['duration']
duration = duration / np.timedelta64(1, 's')
data['duration'] = duration

C:\Users\chbak\AppData\Local\Temp/ipykernel_5376/696655479.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5376\696655479.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_5376/696655479.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5376\696655479.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_nb" was compiled in object mode without forceobj=Tr

In [81]:
#Next event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]

        idx+=1
    return res

data['next_event'] = calculator_nb(data['case'].values, data['event'].values)

C:\Users\chbak\AppData\Local\Temp/ipykernel_5376/1360192189.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5376\1360192189.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_5376/1360192189.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5376\1360192189.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_nb" was compiled in object mode without forceob

In [82]:
#Previous event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx + 1] = event[idx]

        idx+=1
    return res

data['prev_event'] = calculator_nb(data['case'].values, data['event'].values)

C:\Users\chbak\AppData\Local\Temp/ipykernel_5376/1972138414.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5376\1972138414.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_5376/1972138414.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5376\1972138414.py", line 4:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_nb" was compiled in object mode without forceob

In [83]:
#Removing null values
data['next_event'] = data['next_event'].fillna(value='None')
data['prev_event'] = data['prev_event'].fillna(value='None')
data['note'] = data['note'].fillna(value='None')

In [84]:
#unix time
pd.set_option('display.float_format', lambda x: '%.3f' % x)

data['startTime'] = pd.to_datetime(data['startTime'], dayfirst=True)
unixTransform = lambda x: time.mktime(x.timetuple())
data["UNIX_starttime"] = data["startTime"].apply(unixTransform).astype(int)

data['completeTime'] = pd.to_datetime(data['completeTime'], dayfirst=True)
unixTransform = lambda x: time.mktime(x.timetuple())
data["UNIX_completeTime"] = data["completeTime"].apply(unixTransform).astype(int)

#data['REG_DATE'] = pd.to_datetime(data['REG_DATE'], dayfirst=True)
#unixTransform = lambda x: time.mktime(x.timetuple())
#data["UNIX_REG_DATE"] = data["REG_DATE"].apply(unixTransform).astype(int)

#print(data)

In [85]:
#Day of the week
data['weekday'] = data['startTime'].dt.dayofweek

In [115]:
#encoding of categorical data
ordinal_encoder = OrdinalEncoder()
label_encoder = LabelEncoder()

In [87]:
#ensure we have acces to orignal indexing to keep track of the order of events in a process
data['original index'] = data.index

#sorting on time
data.sort_values(by = "UNIX_starttime", ignore_index=True)

,case,event,startTime,completeTime,penalty_JLP1,penalty_JLP3,number_parcels,penalty_JLP2,penalty_JLP5,year,...,docid_uuid,org:resource,success,duration,next_event,prev_event,UNIX_starttime,UNIX_completeTime,weekday,original index
0,980452d16c80c2c3,mail valid,2014-05-04 00:00:00.000,2015-04-24 00:00:00.000,False,False,32,False,False,2015,...,93F2F9CA-6F07-4E87-8107-4852A7B1CBA2,0;n/a,True,30672000.000,mail income,mail valid,1399154400,1429826400,6,63730
1,980452d16c80c2c3,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,32,False,False,2015,...,93F2F9CA-6F07-4E87-8107-4852A7B1CBA2,0;n/a,True,0.000,mail valid,mail valid,1399154400,1399154400,6,63729
2,cb3425ce193199d7,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,5,False,False,2015,...,990E93F0-3BDD-464C-8102-F201E9F5EEE4,0;n/a,True,0.000,mail valid,None,1399154400,1399154400,6,594649
3,980452d16c80c2c3,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,32,False,False,2015,...,2118A637-DCB0-4382-BC47-93BF0CF535AC,0;n/a,True,0.000,mail valid,None,1399154400,1399154400,6,63727
4,cb3425ce193199d7,mail valid,2014-05-04 00:00:00.000,2014-05-04 00:00:00.000,False,False,5,False,False,2015,...,F94BE88D-511C-44B8-B2CE-988A147E7DF8,0;n/a,True,0.000,mail valid,mail valid,1399154400,1399154400,6,594651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514261,8cd07a136f401349,calculate,2018-01-19 12:27:24.824,2018-01-19 12:27:24.824,False,False,37,False,False,2016,...,D555F997-BDE0-471A-99AC-5CC0E1BD650F,75992a,True,0.000,None,begin editing,1516361244,1516361244,4,1020230
2514262,dc56a531f5aeb3b0,save,2018-01-19 12:51:34.000,2018-01-19 12:51:34.000,False,False,49,False,False,2016,...,24CD9E6F-6A0D-4C41-8D76-D3BE35B82681,8d8538,True,0.000,None,finish payment,1516362694,1516362694,4,1553199
2514263,7b405bc5144c14c9,save,2018-01-19 12:57:05.000,2018-01-19 13:02:32.000,False,False,43,False,False,2016,...,6EF39194-5730-4F6E-966C-C6B1D7A7DBC6,8d8538,True,327.000,save,finish payment,1516363025,1516363352,4,1189623
2514264,7b405bc5144c14c9,save,2018-01-19 13:02:32.000,2018-01-19 13:02:32.000,False,False,43,False,False,2016,...,6EF39194-5730-4F6E-966C-C6B1D7A7DBC6,8d8538,True,0.000,None,save,1516363352,1516363352,4,1189624


In [88]:
#separation
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, shuffle=False)

In [89]:
#removing overlap - if case is in both datasets, remove

train_cases = train['case'].unique().tolist()
test_cases = test['case'].unique().tolist()

intersect_list = list(set(train_cases).intersection(test_cases))

In [116]:
#only removes first value in intersect list (needs modification for multiple overlaping values)

#train = train[train['case'] != intersect_list[0]]
#test = test[test['case'] != intersect_list[0]]

#works for more values
org_train = train[:10000].copy()
org_test = test[:10000].copy()
train = train[:10000]
test = test[:10000]
train=train.apply(label_encoder.fit_transform)
test=test.apply(label_encoder.fit_transform)

train = train[train['case'].isin(intersect_list) == False]
X_train_time = train.drop(columns='duration')
Y_train_time = train["duration"]
X_train_event = train.drop(columns=["next_event"])
Y_train_event = train["next_event"]

test = test[test['case'].isin(intersect_list) == False]
X_test_time = test.drop(columns='duration')
Y_test_time = test["duration"]
X_test_event = test.drop(columns=['next_event'])
Y_test_event = test["next_event"]

In [91]:
#separation visualisation

# g = sns.scatterplot(x="UNIX_starttime", y="case", hue="enc_event", data=data, palette='colorblind', legend=False)

#add lines for separation - horizontal and vertical

In [92]:
train.columns

Index(['case', 'event', 'startTime', 'completeTime', 'penalty_JLP1',
       'penalty_JLP3', 'number_parcels', 'penalty_JLP2', 'penalty_JLP5',
       'year', 'penalty_JLP7', 'penalty_JLP6', 'redistribution',
       'amount_applied1', 'amount_applied0', 'amount_applied3',
       'amount_applied2', 'identity:id', 'penalty_V5', 'payment_actual0',
       'payment_actual2', 'payment_actual1', 'penalty_B5F', 'payment_actual3',
       'penalty_B16', 'penalty_GP1', 'basic payment', 'penalty_AGP', 'area',
       'selected_manually', 'penalty_B3', 'penalty_B2', 'selected_risk',
       'penalty_B5', 'penalty_AVBP', 'penalty_B4', 'penalty_B6', 'penalty_ABP',
       'penalty_AVGP', 'penalty_C4', 'greening', 'rejected',
       'cross_compliance', 'penalty_C9', 'penalty_AVJLP', 'penalty_CC',
       'penalty_AVUVP', 'penalty_BGK', 'penalty_C16', 'penalty_BGP',
       'department', 'small farmer', 'risk_factor', 'applicant',
       'penalty_AUVP', 'penalty_amount2', 'penalty_BGKV', 'penalty_amount3',
  

# Feature prediction for time and event based on KBest(z-scores)
note: don't run takes a significant time

In [93]:
from sklearn.feature_selection import SelectKBest

def calc_feature_selection():
    select = SelectKBest(k=10)  # takes best 10 arguments
    z = select.fit_transform(X_train_time, Y_train_time)
    filter = select.get_support()
    print(np.extract(filter, X_train_time.columns))
    #['event' 'year' 'penalty_AVBP' 'penalty_AVGP' 'eventid' 'activity' 'docid''subprocess' 'success' 'next_event']

    select = SelectKBest(k=10)  # takes best 10 arguments
    z = select.fit_transform(X_train_event, Y_train_event)
    filter = select.get_support()
    print(np.extract(filter, X_train_event.columns))
    #['event' 'selected_random' 'note' 'eventid' 'activity' 'subprocess''doctype' 'org:resource' 'duration' 'prev_event']

In [94]:
# calc_feature_selection()

# Naive Baseline Models

In [ ]:
# Naive event (needs restructuring)
data_baseline = org_test[["case", "event", "startTime", "completeTime", "next_event", "enc_event", "original index"]].copy()
#function to add column that keeps track of the position for each event per case
@jit(parallel = True)
def calculator_pos(case):
    res = np.empty(len(case), dtype=object)
    idx = 0
    count=1
    for _ in case:
        if (idx+1 >= len(case)):
            break
        if (case[idx] == case[idx-1]):
            count+=1
            res[idx] = count          
        else:
            count=1
            res[idx]=count
        idx+=1
    res[-1]=count+1
    return res
#get the position of each event per case
data_baseline["pos"] = calculator_pos(data_baseline['case'].to_numpy())
#data_baseline = data_baseline[["case", "event", "startTime", "completeTime", "next_event", "enc_event",  "pos", "original index"]].copy()

#select most occuring event for each position 
#see https://stackoverflow.com/questions/15222754/groupby-pandas-dataframe-and-select-most-common-value
events_count = data_baseline.groupby("pos")['enc_event'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()
events_count = events_count.rename(columns={"pos":"enc_event"})
#Next event for each position (most occuring event in the next position, e.g. 1-25, 2-26, so for 1 we predict 26)
events_count['next_event2'] = events_count['enc_event'].shift(-1)
events_count["next_event2"].iloc[-1] = 0
#map the model results to the original dataframe
data_baseline["next_event2"] = data_baseline["pos"].map(events_count["next_event2"])
#Cleaning, set last events for each case to -1, so that we don't use them in final result and error estimation.
data_baseline["index"] = data_baseline.index
#find last position for each case
last_pos_per_case = data_baseline.groupby("case")[["index","case","pos"]].agg(max)
#use index of the last event per case to assign -1 to the last event
last_pos_per_case.set_index('index', inplace=True)
#last_pos_per_case = last_pos_per_case["next_event2"]
data_baseline.loc[last_pos_per_case.index, "next_event2"] = -1
#Basically, it is the final result of the naive event predictor on the train data
#data_baseline.head(72)

#Error measurement
y_true = data_baseline["enc_event"].to_numpy().astype(int)
y_pred = data_baseline["next_event2"].to_numpy().astype(int)
from sklearn.metrics import f1_score
f1_score(y_true, y_pred, average='micro')

In [ ]:
#Use the model on test dataset

test_baseline = org_train[["case", "event", "startTime", "completeTime", "next_event", "enc_event", "original index"]].copy()
# get position column for the test dataset (required for the model to operate)
test_baseline["pos"] = calculator_pos(test_baseline['case'].to_numpy())
#drop unnecessary stuff to speed up the process
test_baseline = test_baseline[["case", "event", "startTime", "completeTime", "next_event", "enc_event",  "pos", "original index"]].copy()
#map the model results (obrained from train dataset) to the original dataframe
test_baseline["next_event2"] = test_baseline["pos"].map(events_count["next_event2"])

#Cleaning, set last events for each case to -1, so that we don't use them in final result and error estimation.
test_baseline["index"] = test_baseline.index
#find last position for each case
last_pos_per_case = test_baseline.groupby("case")[["index","case","pos"]].agg(max)#.to_frame()
#use index of the last event per case to assign -1 to the last event
last_pos_per_case.set_index('index', inplace=True)
#last_pos_per_case = last_pos_per_case["next_event2"]
test_baseline.loc[last_pos_per_case.index, "next_event2"] = -1
#Basically, it is the final result of the naive event predictor on the test data
test_baseline.head(72)

#Error measurement
y_true = test_baseline["enc_event"].to_numpy().astype(int)
y_pred = test_baseline["next_event2"].to_numpy().astype(int)
#y_true[-1] = 0
#y_true, y_pred
from sklearn.metrics import f1_score
f1_score(y_true, y_pred, average='micro')

In [ ]:
# Naive time predictor (wasn't tested on test dataset, since I have no idea how to measure time accuracy)

#find difference between completeTime of current event and the next event
data_baseline["duration_start-start"] = data_baseline["startTime"].diff()
#shift it up, so the difference corresponds to the current event
data_baseline["duration_start-start"] = data_baseline["duration_start-start"].shift(-1)
#set time duration between cases to NaT (last event per case, last position), since we only want time duration per case
data_baseline.loc[ data_baseline[ data_baseline["next_event2"] == -1 ].index, "duration_start-start"] = 'NaT'
#find average time duration between startTime of the events at current position and at the next position
predicted_duration = data_baseline.groupby("pos")['duration_start-start'].agg('mean')
#map the model results to the test dataset
data_baseline["predicted_duration"] = data_baseline['pos'].map(predicted_duration)
#add predicted duration to completeTime to predict the startTime of the event in the next position
data_baseline["predicted_time"] = data_baseline["startTime"] + data_baseline["predicted_duration"]
data_baseline = data_baseline.drop(['predicted_duration', 'duration_start-start'], axis=1)
#data_baseline

Previous naive predictor

In [95]:
# Naive event (needs restructuring)
data_baseline= test.copy()

@jit(parallel = True)
def calculator_pos(case):
    res = np.empty(len(case), dtype=object)
    idx = 0
    count=1
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx] == case[idx-1]):
            count+=1
            res[idx] = count
            
        else:
            count=1
            res[idx]=count

        idx+=1
    res[-1]=count+1
    return res

data_baseline["pos"] = calculator_pos(data_baseline['case'].values)

event_to_num = {}
list_of_events = train["event"].unique()
i=0
for event in list_of_events:
    event_to_num[str(event)] = i
    i += 1
event_to_num['None'] = i

pop=data_baseline.sort_values(by='pos')
pop['eventnum']=pop['event']
pop2=pop.set_index('pos')
pop3=pop[['pos','eventnum']]
pop4=pop3.groupby(['pos', 'eventnum']).apply(pd.DataFrame.mode).reset_index(drop=True)
pop5=pop4.drop_duplicates(subset='pos')
ptenum= dict(zip(pop5.pos, pop5.eventnum))
num_to_event = {value:key for key, value in event_to_num.items()}
data_baseline['predicted_event_num'] = (data_baseline['pos']+1).map(ptenum)
data_baseline['predicted_event'] = (data_baseline['predicted_event_num']).map(num_to_event)
data_baseline_final=data_baseline.drop(['predicted_event_num'],axis=1)

next_task=[]
predicted_event=[]
for event in data_baseline_final['next_event']:
    next_task.append(str(event))
    

for case in data_baseline_final['predicted_event']:
    predicted_event.append(str(case))

accuracy_score(next_task,predicted_event)


test["naive_event"] = predicted_event

C:\Users\chbak\AppData\Local\Temp/ipykernel_5376/1978921505.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_pos" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5376\1978921505.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_5376/1978921505.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_pos" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5376\1978921505.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_pos" was compiled in object mode without forc

In [96]:
# Naive time
#Sums up count for each event and the time each event takes
events_count = train.groupby("event")['duration'].agg('count')
event_duration_sum = train.groupby("event")['duration'].agg('sum')

#Computes average duration per event (basically our trained data that can be mapped onto test data)
duration_per_event = event_duration_sum / events_count 

test["naive_time"] = test['event'].map(duration_per_event)

# Random Forest

In [97]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def event_metrics(y_test, y_pred, avg="weighted", model="..."):
 
    prec_score = precision_score(y_test, y_pred, average=avg, zero_division=0)
    rec_score = recall_score(y_test, y_pred, average=avg, zero_division=0)
    F1_score = f1_score(y_test, y_pred, average=avg, zero_division=0)
    acc_score = accuracy_score(y_test, y_pred)

    print("\n")
    print(f"Error metrics for the {model} event model")
    print("\n")
    print(f'The accuracy of the model is {acc_score}.')
    print(f'The precision of the model is {prec_score}, using {avg} average.')
    print(f'The recall of the model is {rec_score}, using {avg} average.')
    print(f'The f1-score of the model is {F1_score}, using {avg} average.')
    print("\n")

    return acc_score, prec_score, rec_score, F1_score

In [98]:
def calc_random_forest():
    # Create the random grid

    RF = RandomForestClassifier(n_estimators=300,
                                min_samples_split=10,
                                min_samples_leaf=2,
                                max_features='sqrt',
                                max_depth=50,
                                bootstrap=True)

    dataset_col = [
        'event',
        'selected_random',
        'subprocess',
        'org:resource',
        'duration',
        'prev_event',
    ]

    RF_fit = RF.fit(X_train_event.filter(items=dataset_col), Y_train_event)
    RF_pred = RF_fit.predict(X_test_event.filter(items=dataset_col))
    # org_test["event_RF"] = RF_pred
    event_metrics(Y_test_event, RF_pred, model="Random Forest") #91% accuracy on testing

calc_random_forest()



Error metrics for the Random Forest event model


The accuracy of the model is 0.3032.
The precision of the model is 0.3467355006100965, using weighted average.
The recall of the model is 0.3032, using weighted average.
The f1-score of the model is 0.3146013246570676, using weighted average.




# LSTM

In [99]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation

In [100]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def time_metrics(y_test, y_pred, model="..."):
  
    print("\n")
    print(f"Error metrics for the {model} time model")
    print("\n")
    print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
    print('$R_2$ score:', r2_score(y_test, y_pred))
    print("\n")

In [101]:
def calc_LSTM():
    listVal = train
    columnNames = [
        'event', 'penalty_AVBP', 'penalty_AVGP', 'eventid', 'activity',
        'docid', 'subprocess', 'success', 'next_event'
    ]
    listValSelected = listVal[columnNames]
    listValSelected_prediction = test[columnNames]
    listValSelected_prediction = listValSelected_prediction.values
    listValDuration_prediction = org_test['duration']
    listValDuration_prediction = listValDuration_prediction.values
    listValDuration = org_train['duration'].values

    listValSelected = listValSelected.values
    n_steps = len(listValSelected[0])
    # split into samples
    n_features = 1
    X = listValSelected.reshape(
        (listValSelected.shape[0], listValSelected.shape[1], n_features))
    y = listValDuration

    # define model
    model = Sequential()
    model.add(
        LSTM(
            100,
            input_shape=(n_steps, n_features),
            #  stateful=True,
            return_sequences=False))
    model.add(Dropout(0.2))
    # model.add(LSTM(units=100))
    # model.add(Dropout(0.2))
    # model.add(Dense(units=1))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam', loss='mse')

    model.fit(X,
              y,
              batch_size=100,
              epochs=1,
              verbose=1,
              workers=-1,
              use_multiprocessing=True)

    # demonstrate prediction
    x_input = listValSelected_prediction

    x_input = x_input.reshape((x_input.shape[0], n_steps, n_features))
    yhat = model.predict(x_input, verbose=1, use_multiprocessing=True)
    org_test["time_LSTM"] = yhat.flatten()[:len(listValDuration_prediction)]
    time_metrics(listValDuration_prediction, yhat.flatten()[:len(listValDuration_prediction)], "LSTM")
calc_LSTM()

313/313 [==============================] - 2s 5ms/step


Error metrics for the LSTM time model


Mean Absolute Error: 524297.3315508114
Mean Squared Error: 3884620811392.472
Root Mean Squared Error: 1970944.1421289626
$R_2$ score: -0.07615180404045385




<h1>Neural network</h1>

In [102]:
data = data[:10000]

In [103]:
def normalize(df_name, col_name):
    col_as_array = df_name[col_name].to_numpy()
    col_as_array = np.where(col_as_array == 0, 0.01, col_as_array)
    col_as_array_norm = np.log10(col_as_array)
    mean = col_as_array_norm.mean()
    stdev = col_as_array_norm.std()
    epsilon = 0.01
    return (col_as_array_norm - mean) / (stdev + epsilon)

In [104]:
def prepfeatures(df_name):
    event = df_name['event'].to_numpy()
    event = event.reshape(-1,1)
    event = ordinal_encoder.fit_transform(event)
    
    #selected_random = df_name['selected_random'].to_numpy()
    #selected_random = selected_random.reshape(-1,1)
    #selected_random = ordinal_encoder.fit_transform(selected_random)
    
    #note = df_name['note'].to_numpy()
    #note = note.reshape(-1,1)
    #note = ordinal_encoder.fit_transform(note)
    
    #eventid = org_train['eventid'].to_numpy()
    #eventid = eventid + abs(org_train['eventid'].min())
    
    #subprocess = df_name['subprocess'].to_numpy()
    #subprocess = subprocess.reshape(-1,1)
    #subprocess = ordinal_encoder.fit_transform(subprocess)
    
    #doctype = df_name['doctype'].to_numpy()
    #doctype = doctype.reshape(-1,1)
    #doctype = ordinal_encoder.fit_transform(doctype)
    
    duration = normalize(df_name,'duration')
    weekday = df_name['weekday'].to_numpy()
    
    #startTime = normalize(df_name,'UNIX_starttime')
    
    prev_event = df_name['prev_event'].to_numpy()
    prev_event = prev_event.reshape(-1,1)
    prev_event = ordinal_encoder.fit_transform(prev_event)
    
    features = []
    for i in range(len(event)):
        current = event[i]
        #current = np.append(current,selected_random[i])
        #current = np.append(current,note[i])
        #current = np.append(current,eventid[i])
        #current = np.append(current,subprocess[i])
        #current = np.append(current,doctype[i])
        current = np.append(current,duration[i])
        current = np.append(current,weekday[i])
        #current = np.append(current,startTime[i])
        current = np.append(current,prev_event[i])
        features.append(current)
        
    return np.array(features)

In [105]:
def preplabels(df_name):
    labels = df_name['next_event'].to_numpy()
    labels = label_encoder.fit_transform(labels)
    labels = labels.reshape(-1, 1)
    
    return np.array(labels)

In [106]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(4,)),
    keras.layers.Dense(20, activation='softplus'),
    keras.layers.Dropout(1/10),
    keras.layers.Dense(30, activation='softplus'),
    keras.layers.Dropout(1/15),
    keras.layers.Dense(42, activation='softplus')
])

model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [107]:
losses = []
accuracies = []

def crossvalidation(k):
    quantile = int(np.floor(len(data)/5))
    for i in range(1,k-1):
        train = data[:(quantile*i)]
        train = train.sample(frac=1)
        test = data[(quantile*i):(quantile*(i+1))]
        
        features = prepfeatures(train)
        labels = preplabels(train)
        
        print("Training on 0:",(quantile*i),"; Testing on ",(quantile*i),":",(quantile*(i+1)))
        model.fit(features,labels,epochs=3,verbose=1)
        
        features_test = prepfeatures(test)
        labels_test = preplabels(test)
        
        eval = model.evaluate(features_test,labels_test)
        losses.append(eval[0])
        accuracies.append(eval[1])
        
    return losses, accuracies

In [108]:
losses, accuracies = crossvalidation(5)

Training on 0: 2000 ; Testing on  2000 : 4000
Epoch 1/3
63/63 [==============================] - 0s 2ms/step - loss: 3.8545 - accuracy: 0.0640
Epoch 2/3
63/63 [==============================] - 0s 2ms/step - loss: 2.6546 - accuracy: 0.2185
Epoch 3/3
63/63 [==============================] - 0s 2ms/step - loss: 4.2573 - accuracy: 0.0040
Training on 0: 4000 ; Testing on  4000 : 6000
Epoch 1/3
125/125 [==============================] - 1s 4ms/step - loss: 3.1395 - accuracy: 0.2295
Epoch 2/3
125/125 [==============================] - 0s 3ms/step - loss: 2.3880 - accuracy: 0.3077
Epoch 3/3
63/63 [==============================] - 0s 2ms/step - loss: 4.8526 - accuracy: 0.0025
Training on 0: 6000 ; Testing on  6000 : 8000
Epoch 1/3
188/188 [==============================] - 0s 2ms/step - loss: 2.4072 - accuracy: 0.2268
Epoch 2/3
188/188 [==============================] - 0s 2ms/step - loss: 2.0792 - accuracy: 0.2835
Epoch 3/3
63/63 [==============================] - 0s 2ms/step - loss: 5.5893 

In [109]:
np.array(accuracies).mean()

0.011666666716337204

In [110]:
features_data = prepfeatures(data)
prediction = model.predict(features_data)

In [111]:
predicted_events = []
for i in range(len(prediction)):
    predicted_events.append(np.argmax(prediction[i]))

In [117]:
predicted_events = label_encoder.inverse_transform(predicted_events)

In [118]:
data['neuralnet_event_prediction'] = predicted_events

# Regression

In [119]:
def prepfeatures_regression(df_name):
    event = df_name['event'].to_numpy()
    event = event.reshape(-1,1)
    event = ordinal_encoder.fit_transform(event)
    
    next_event = df_name['next_event'].to_numpy()
    next_event = next_event.reshape(-1,1)
    next_event = ordinal_encoder.fit_transform(next_event)
    
    year = df_name['year'].to_numpy()
    year = year.reshape(-1,1)
    year = ordinal_encoder.fit_transform(year)
    
    penalty_AVBP = df_name['penalty_AVBP'].to_numpy()
    penalty_AVBP = penalty_AVBP.reshape(-1,1)
    penalty_AVBP = ordinal_encoder.fit_transform(penalty_AVBP)
    
    penalty_AVGP = df_name['penalty_AVGP'].to_numpy()
    penalty_AVGP = penalty_AVGP.reshape(-1,1)
    penalty_AVGP = ordinal_encoder.fit_transform(penalty_AVGP)
    
    success = df_name['success'].to_numpy()
    success = success.reshape(-1,1)
    success = ordinal_encoder.fit_transform(success)
    
    eventid = df_name['eventid'].to_numpy()
    eventid = success.reshape(-1,1)
    
    docid = df_name['docid'].to_numpy()
    docid = success.reshape(-1,1)
    
    subprocess = df_name['subprocess'].to_numpy()
    subprocess = subprocess.reshape(-1,1)
    subprocess = ordinal_encoder.fit_transform(subprocess)
    
    weekday = df_name['weekday'].to_numpy()
    weekday = weekday.reshape(-1,1)
    
    X = []
    for i in range(len(event)):
        current = event[i]
        current = np.append(current, year[i])
        current = np.append(current, penalty_AVBP[i])
        current = np.append(current, penalty_AVGP[i])
        current = np.append(current, success[i])
        current = np.append(current, eventid[i])
        current = np.append(current, docid[i])
        current = np.append(current, next_event[i])
        current = np.append(current, subprocess[i])
        current = np.append(current, weekday[i])
        X.append(current)
        
    return np.array(X, dtype=float)

In [120]:
def preplabels_regression(df_name):
    duration = df_name['duration'].to_numpy()
    return np.array(duration, dtype=float)

In [121]:
X = prepfeatures_regression(org_train)
y = preplabels_regression(org_train)

In [122]:
huber = HuberRegressor().fit(X, y)

X_test = prepfeatures_regression(org_test)

org_test['regression_duration'] = huber.predict(X_test)
org_test['error'] = np.absolute(org_test['duration'] - org_test['regression_duration'])
org_test['error'].mean()

2086.232180857761

In [123]:
X_for_prediction = prepfeatures_regression(data)
data['regression_time_prediction'] = huber.predict(X_for_prediction)
data['regression_time_prediction'] = data['regression_time_prediction'] + data['UNIX_starttime']

In [124]:
data['regression_time_predicition'] = data['regression_time_prediction'].apply(datetime.fromtimestamp)